<a href="https://colab.research.google.com/github/gevenbly/ML_test/blob/main/ML_bin_functs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import random

In [1]:
def group_to_isometry(chi_locs):
  
  # construct optimal isometry from groupings
  d0 = sum([len(item) for item in chi_locs])
  chi_temp = len(chi_locs)
  wiso = np.zeros((d0, chi_temp), dtype=int)
  for k in range(chi_temp):
    wiso[chi_locs[k],k] = 1

  return wiso

In [ ]:
def optimal_grouping(gam_block, chi_max):

  ### find spanning set for optimal isometry
  d0 = gam_block.shape[0]
  if d0 < chi_max:
    chi_locs = [0] * d0
    for k in range(d0):
      chi_locs[k] = [k]

  else:
    ## final optimal split for single indices

    # initialize arrays
    chi_kept = []
    chi_locs = [0] * chi_max
    chi_rem = np.arange(d0)
    for k in range(chi_max):
      chi_locs[k] = []

    # find most densely populated row
    loc = np.argmax(np.sum(gam_block, axis=0))
    chi_kept.append(loc)
    chi_locs[0].append(loc)
    chi_rem = chi_rem[chi_rem != loc]

    for k in range(1,chi_max):
      # find most distinguishable rows
      min_vals = np.min(gam_block[chi_kept,:][:,chi_rem], axis=0)
      max_all = np.max(min_vals)
      max_locs = np.where(min_vals == max_all)[0]

      # in case of degeneracy of max row, sort by total overlap
      sum_vals = np.sum(gam_block[chi_kept,:][:,chi_rem].reshape(len(chi_kept), len(chi_rem)), axis=0)
      final_loc = chi_rem[max_locs[np.argmax(sum_vals[max_locs])]]

      chi_kept.append(final_loc)
      chi_locs[k].append(final_loc)
      chi_rem = chi_rem[chi_rem != final_loc]

    ## final optimal group for remaining indices

    for k in range(len(chi_rem)):
      
      # choose which index to focus on 
      min_vals = np.min(gam_block[chi_kept,:][:,chi_rem], axis=0)
      min_all = np.min(min_vals)
      min_locs = np.where(min_vals == min_all)[0]

      sum_vals = np.sum(gam_block[chi_kept,:][:,chi_rem].reshape(len(chi_kept), len(chi_rem)), axis=0)
      final_loc = chi_rem[min_locs[np.argmax(sum_vals[min_locs])]]

      # find optimal group for focused index
      tot_over = np.zeros(chi_max, dtype=int)
      for p in range(chi_max):
        tot_over[p] = np.sum(gam_block[chi_locs[p],final_loc])

      min_over = np.min(tot_over)
      locs = np.where(tot_over == min_over)[0]
      final_group = locs[np.random.randint(len(locs))]

      chi_kept.append(final_loc)
      chi_locs[final_group].append(final_loc)
      chi_rem = chi_rem[chi_rem != final_loc]

  return chi_locs



In [ ]:
def make_disting(rho_off, phi_AB, label_locs):

  ### make distinguishablility matrix
  L = len(phi_AB)
  d = phi_AB[0].shape[1]

  valid_locs = np.where(label_locs[:,0])[0]
  gam_block = np.zeros((d, d), dtype=int)
  for k in valid_locs:
    locx = label_locs[k,1]
    locy = label_locs[k,2]
    gam_block += (phi_AB[locx].T) @ rho_off[k] @ phi_AB[locy]

  gam_block += gam_block.T

  return gam_block

In [ ]:
def make_overlaps(phi_samples, sites):

  ### left block for distinguishablility matrix
  L = len(phi_samples[0])

  # environment sites
  rho_off = [0] * (L**2)
  label_locs = np.zeros((L**2, 3), dtype = int)
  
  for k in range(L):
    for p in range(k):
      Mk = phi_samples[0][k].shape[0]
      Mp = phi_samples[0][p].shape[0]
      rho_off[k*L + p] = np.ones((Mk,Mp), dtype=int)
      label_locs[k*L + p,:] = [1,k,p]
      for r in sites:
        if r < len(phi_samples) and r >= 0:
          rho_off[k*L + p] = np.multiply(rho_off[k*L + p], 
                                        (phi_samples[r][k] @ (phi_samples[r][p].T)))

  return rho_off, label_locs

In [ ]:
def comb_blocks(phi_A, phi_B):
  # make block sites
  L = len(phi_A)
  phi_AB = [0] * L
  for k in range(L):
    Mk = phi_A[k].shape[0]
    d0 = phi_A[k].shape[1]
    d1 = phi_B[k].shape[1]

    phi_AB[k] = np.einsum("ij,ik->ijk",
                          phi_A[k], phi_B[k]
                          ).reshape(Mk, d0*d1) 
  
  return phi_AB

In [ ]:
def evaluate_final(phi_samples0):

  L = len(phi_samples0)

  total_in_cater = np.zeros((L,L), dtype=int)
  for k in range(L):
    total_in_cater[k,:] = np.sum(phi_samples0[k], axis=0)

  lab_remain = np.arange(L)
  lab_loc = np.zeros(L, dtype=int)
  tot_correct = 0
  for k in range(L):

    lab_loc[k] = lab_remain[np.argmax(total_in_cater[lab_remain,k])]
    lab_remain = lab_remain[lab_remain != lab_loc[k]]
    tot_correct += total_in_cater[lab_loc[k],k]

  return tot_correct

In [ ]:
def triple_site_block(phi_trip, chi_max, left_side=True, alp_weight=1):

  s0 = phi_trip.shape
  if left_side:
    phi_trip = phi_trip.reshape(s0[0]*s0[1],s0[2],s0[3])  
  else:
    phi_trip = phi_trip.reshape(s0[0],s0[1]*s0[2],s0[3]).transpose(1,0,2)  
    # phi_trip = phi_trip.transpose(0,2,1,3).reshape(s0[0],s0[1]*s0[2],s0[3]).transpose(1,0,2)  
    
  d0 = phi_trip.shape[0]
  chi_locs = [0] * (d0)
  for k in range(d0):
    chi_locs[k] = [k]

  corr0 = np.sum(np.sum(np.sum(phi_trip, axis=2) - np.max(phi_trip, axis=2), axis=1))

  for p in range(d0 - chi_max):
    dtemp = phi_trip.shape[0]

    overlap_mat = np.zeros((dtemp,dtemp), dtype=int)
    common_mat = np.zeros((dtemp,dtemp), dtype=int)
    wrong_vec = np.sum(np.sum(phi_trip, axis=2) - np.max(phi_trip, axis=2), axis=1)

    sum_mat = np.zeros((dtemp,dtemp), dtype=int)
    sum_vec = np.sum(phi_trip, axis=2)
    for k in range(dtemp):
      phi_temp = phi_trip[k,:,:] + phi_trip
      overlap_mat[:,k] = np.sum(np.sum(phi_temp, axis=2) - np.max(phi_temp, axis=2), axis=1) - (wrong_vec[k] + wrong_vec)
      common_mat[:,k] = np.sum((phi_trip + phi_trip[k,:,:]) - np.abs(phi_trip - phi_trip[k,:,:]), axis=(1,2))

      new_shape = list(sum_vec.shape) + [1]
      sum_mat[:,k] = np.sum(np.min(np.concatenate(
          [np.tile(sum_vec[k,:], phi_trip.shape[0]).reshape(new_shape),sum_vec.reshape(new_shape)],
          axis=2).reshape(phi_trip.shape[0],phi_trip.shape[1],2), axis=2), axis=1)

    # print(np.trace(overlap_mat))
    merit_mat = alp_weight*overlap_mat - sum_mat
    all_max = np.max(np.abs(merit_mat)) + 1
    merit_mat -= np.diag(np.diag(merit_mat))
    merit_mat += np.diag(all_max * np.ones(dtemp, dtype=int)) 
    # if (np.min(merit_mat) == 0):
    #   break

    x0, x1 = np.divmod(np.argmin(merit_mat), dtemp)
    y0 = min(x0,x1)
    y1 = max(x0,x1)
    # print([x0, x1])

    chi_locs[y0] += chi_locs.pop(y1)
    phi_trip[y0,:,:] += phi_trip[y1,:,:]
    phi_trip = np.delete(phi_trip, y1, axis=0)



    # print(chi_locs)
    # x = [item for sublist in chi_locs for item in sublist]
    if (x0==x1):
      print(merit_mat)
      print([x0,x1])
      print([chi_locs])
    # print(len(x))

  corr1 = np.sum(np.sum(np.sum(phi_trip, axis=2) - np.max(phi_trip, axis=2), axis=1))
  print("hello",[corr0,corr1])

  # print(chi_locs)
  return group_to_isometry(chi_locs), chi_locs

In [ ]:
def make_trip_state(phi_left, phi_samples, phi_right):

  L = len(phi_left)
  dL = phi_left[0].shape[1]
  dM = phi_samples[0].shape[1]
  dR = phi_right[0].shape[1]

  phi_trip = np.zeros((dL,dM,dR,L), dtype=int)
  for k in range(L):
    phi_trip[:,:,:,k] = np.einsum("ij,ik,il->jkl",phi_left[k], phi_samples[k], 
                            phi_right[k]).reshape(dL,dM,dR)

  return(phi_trip)

In [ ]:
def make_left_iso(phi_left, phi_mid, phi_right, chi_max):

  dL = phi_left[0].shape[1]
  d = phi_mid[0].shape[1]
  dR = phi_right[0].shape[1]

  phi_trip = np.zeros((dL*d,dR,L), dtype=int)
  for k in range(L):
    phi_trip[:,:,k] = np.einsum("ij,ik,il->jkl",phi_left[k], phi_mid[k], 
                            phi_right[k]).reshape(dL*d,dR)

  gam_mat = np.zeros((dL*d,dL*d), dtype=int)
  # dis_mat = np.zeros((dL*d,dL*d), dtype=int)
  for k0 in range(L):
    for k1 in range(L):
      if k0==k1:
        # gam_mat += phi_trip[:,:,k0] @ phi_trip[:,:,k1].T
        x = 0
      else:
        gam_mat -= phi_trip[:,:,k0] @ phi_trip[:,:,k1].T
        # dis_mat += phi_trip[:,:,k0] @ phi_trip[:,:,k1].T

  chi_locs = [0] * (d*dL)
  for k in range(d*dL):
    chi_locs[k] = [k]

  for k in range(dL*d - chi_max):
    dtemp = gam_mat.shape[0]
    join_weight = np.sum(gam_mat)*np.ones((dtemp,dtemp), dtype=int)
    for f0 in range(dtemp):
      for f1 in range(f0):
        # join_weight[f0,f1] = gam_mat[f0,f0] + gam_mat[f1,f1] + gam_mat[f0,f1] + gam_mat[f1,f0]
        join_weight[f0,f1] = gam_mat[f0,f1] + gam_mat[f1,f0]

    x0, x1 = np.divmod(np.argmax(join_weight), dtemp)
    # temp_max = np.max(np.abs(join_weight))
    # exp_weight = np.exp(1*join_weight / temp_max)
    # exp_weight = exp_weight - np.diag(np.diag(exp_weight))
    # exp_weight = exp_weight / np.sum(exp_weight)

    # temp_sum = np.sum(exp_weight)
    # temp_val = temp_sum*np.random.rand()
    # cs_weight = np.cumsum(exp_weight)
    # temp_loc = dtemp**2 - np.sum(cs_weight > temp_val) 
    # x0, x1 = np.divmod(temp_loc, dtemp)
    # if (x0 == x1):
    #   x0 -= 1
    #   print(xxx)

    chi_locs[x1] += chi_locs.pop(x0)
    gam_mat[:,x1] += gam_mat[:,x0]
    gam_mat[x1,:] += gam_mat[x0,:]
    gam_mat = np.delete(gam_mat, x0, axis=0)
    gam_mat = np.delete(gam_mat, x0, axis=1)

    

  # print(chi_locs)
  w_iso = group_to_isometry(chi_locs)
  return w_iso

In [ ]:
def make_right_iso(phi_left, phi_mid, phi_right, chi_max):

  dL = phi_left[0].shape[1]
  d = phi_mid[0].shape[1]
  dR = phi_right[0].shape[1]
  
  phi_trip = np.zeros((dL,dR*d,L), dtype=int)
  for k in range(L):
    phi_trip[:,:,k] = np.einsum("ij,ik,il->jkl",phi_left[k], phi_mid[k], 
                            phi_right[k]).reshape((dL,dR*d))

  gam_mat = np.zeros((dR*d,dR*d), dtype=int)
  for k0 in range(L):
    for k1 in range(L):
      if k0==k1:
        # gam_mat += phi_trip[:,:,k0].T @ phi_trip[:,:,k1]
        x = 0
      else:
        gam_mat -= phi_trip[:,:,k0].T @ phi_trip[:,:,k1]

  chi_locs = [0] * (d*dR)
  for k in range(d*dR):
    chi_locs[k] = [k]

  for k in range(dR*d - chi_max):
    dtemp = gam_mat.shape[0]
    join_weight = np.sum(gam_mat)*np.ones((dtemp,dtemp), dtype=int)
    for f0 in range(dtemp):
      for f1 in range(f0):
        # join_weight[f0,f1] = gam_mat[f0,f0] + gam_mat[f1,f1] + gam_mat[f0,f1] + gam_mat[f1,f0]
        join_weight[f0,f1] = gam_mat[f0,f1] + gam_mat[f1,f0]

    x0, x1 = np.divmod(np.argmax(join_weight), dtemp)
    # j0 = join_weight[x0,x1]

    # temp_max = np.max(np.abs(join_weight))
    # exp_weight = np.exp(1*join_weight / temp_max)
    # exp_weight = exp_weight - np.diag(np.diag(exp_weight))
    # exp_weight = exp_weight / np.sum(exp_weight)

    # temp_sum = np.sum(exp_weight)
    # temp_val = temp_sum*np.random.rand()
    # cs_weight = np.cumsum(exp_weight)
    # temp_loc = dtemp**2 - np.sum(cs_weight > temp_val) 
    # x0, x1 = np.divmod(temp_loc, dtemp)
    # if (x0 == x1):
    #   x0 -= 1
    #   print(xxx)
    
    chi_locs[x1] += chi_locs.pop(x0)

    gam_mat[:,x1] += gam_mat[:,x0]
    gam_mat[x1,:] += gam_mat[x0,:]
    gam_mat = np.delete(gam_mat, x0, axis=0)
    gam_mat = np.delete(gam_mat, x0, axis=1)

  # print(chi_locs)
  w_iso = group_to_isometry(chi_locs)
  return w_iso